In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import math
import missingno as msno

In [2]:
df = pd.read_csv("small_data/trimmed_data.csv")
df_orig = pd.read_csv("small_data/3000_training_set_VU_DM.csv")

In [4]:
for var in df:
    col = df[var]
    missing = col.isnull().values.sum()
    if missing >0:
        print('the number is missing values in', var, 'is', col.isnull().values.sum())


the number is missing values in visitor_hist_starrating is 2800
the number is missing values in visitor_hist_adr_usd is 2800
the number is missing values in prop_review_score is 2
the number is missing values in prop_location_score2 is 674
the number is missing values in prop_log_historical_price is 254
the number is missing values in srch_query_affinity_score is 2897
the number is missing values in orig_destination_distance is 928
the number is missing values in comp1_rate is 2900
the number is missing values in comp1_inv is 2893
the number is missing values in comp1_rate_percent_diff is 2919
the number is missing values in comp2_rate is 1790
the number is missing values in comp2_inv is 1731
the number is missing values in comp2_rate_percent_diff is 2701
the number is missing values in comp3_rate is 1966
the number is missing values in comp3_inv is 1905
the number is missing values in comp3_rate_percent_diff is 2745
the number is missing values in comp4_rate is 2866
the number is miss

In [5]:

# Code to replace missing prop_review_score values with corresponding propstarrating vlaues.
def fill_prop_review_score(df):
    for i in range(len(df)):
        if math.isnan(df['prop_review_score'][i]):
            df['prop_review_score'][i] = df['prop_starrating'][i]
    return(df)

fill_prop_review_score(df)

# Dorp column 'prop_location_score2', too many missing values.
df.drop(columns=['prop_location_score2'])

<ipython-input-5-41b6787c7c68>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prop_review_score'][i] = df['prop_starrating'][i]


,Unnamed: 0,srch_id,month,daypart,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,...,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd
0,0,1,4,Morning,187,NaN,NaN,219,893,3,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN
1,1,1,4,Morning,187,NaN,NaN,219,10404,4,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN
2,2,1,4,Morning,187,NaN,NaN,219,21315,3,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN
3,3,1,4,Morning,187,NaN,NaN,219,27348,2,...,NaN,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN
4,4,1,4,Morning,187,NaN,NaN,219,29604,4,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,191,2,Noon,55,NaN,NaN,55,1131,1,...,NaN,NaN,-1.0,0.0,2.0,NaN,NaN,NaN,0,NaN
2996,2996,191,2,Noon,55,NaN,NaN,55,2762,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2997,2997,191,2,Noon,55,NaN,NaN,55,12958,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2998,2998,191,2,Noon,55,NaN,NaN,55,36006,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [25]:

df_orig = pd.read_csv("small_data/3000_training_set_VU_DM.csv")


# Code which adds columns with the average rate,inventory, and percentage difference between Expedia and competitors.
def make_rates(data):
    data['comp_rate'] = data[['comp1_rate',
                            'comp2_rate',
                            'comp3_rate',
                            'comp4_rate',
                            'comp5_rate',
                            'comp6_rate',
                            'comp7_rate',
                            'comp8_rate']].mean(axis=1)
    data['comp_rate'] = data['comp_rate'].fillna(0)
    
    data['comp_inv'] = data[['comp1_inv',
                            'comp2_inv',
                            'comp3_inv',
                            'comp4_inv',
                            'comp5_inv',
                            'comp6_inv',
                            'comp7_inv',
                            'comp8_inv']].mean(axis=1)
    data['comp_inv'] = data['comp_inv'].fillna(0)
    
    data['comp_percent'] = data[['comp1_rate_percent_diff',
                            'comp2_rate_percent_diff',
                            'comp3_rate_percent_diff',
                            'comp4_rate_percent_diff',
                            'comp5_rate_percent_diff',
                            'comp6_rate_percent_diff',
                            'comp7_rate_percent_diff',
                            'comp8_rate_percent_diff']].mean(axis=1)
    data['comp_percent'] = data['comp_percent'].fillna(0)
    
    return(data)
make_rates(df_orig)

,Unnamed: 0,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,...,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,comp_rate,comp_inv,comp_percent
0,0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,...,NaN,0.0,0.0,NaN,0,NaN,0,0.00,0.000000,0.0
1,1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,...,NaN,0.0,0.0,NaN,0,NaN,0,0.00,0.333333,0.0
2,2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,...,NaN,0.0,0.0,NaN,0,NaN,0,0.00,0.000000,0.0
3,3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,...,NaN,-1.0,0.0,5.0,0,NaN,0,-0.75,0.250000,5.0
4,4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,...,NaN,0.0,0.0,NaN,0,NaN,0,0.00,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2995,191,2013-02-07 20:58:47,15,55,NaN,NaN,55,1131,1,...,2.0,NaN,NaN,NaN,0,NaN,0,-0.50,0.000000,2.0
2996,2996,191,2013-02-07 20:58:47,15,55,NaN,NaN,55,2762,2,...,NaN,NaN,NaN,NaN,0,NaN,0,-1.00,0.000000,29.0
2997,2997,191,2013-02-07 20:58:47,15,55,NaN,NaN,55,12958,3,...,NaN,NaN,NaN,NaN,0,NaN,0,0.00,0.000000,0.0
2998,2998,191,2013-02-07 20:58:47,15,55,NaN,NaN,55,36006,3,...,NaN,NaN,NaN,NaN,0,NaN,0,0.00,0.000000,0.0
